In [1]:
import sys
sys.path.append("..")
from lib.kalman_gst import *  
from pygsti.modelpacks import smq1Q_XYI as std

In [2]:
from pygsti.forwardsims import MatrixForwardSimulator

In [3]:
SAMPLES = 256

In [4]:
# setup the FOGI model
mdl_datagen = std.target_model('H+s')
basis1q = pygsti.baseobjs.Basis.cast('pp', 4)
gauge_basis = pygsti.baseobjs.CompleteElementaryErrorgenBasis(
                        basis1q, mdl_datagen.state_space, elementary_errorgen_types='HS')
mdl_datagen.setup_fogi(gauge_basis, None, None, reparameterize=True,
                     dependent_fogi_action='drop', include_spam=True)
target_model = mdl_datagen.copy()

In [5]:
tfs = MatrixForwardSimulator(target_model)

In [6]:
mdl_datagen.num_params

18

In [218]:
hamiltonian_params = [0, 1, 2, 6, 9]
print([mdl_datagen.fogi_errorgen_component_labels()[i] for i in hamiltonian_params])

['H(X:0)_[]', 'H(Y:0)_[]', 'H(Z:0)_[]', 'H(X:0)_Gxpi2:0', 'H(Y:0)_Gypi2:0']


In [219]:
# add noise to the stochastic and hamiltonian parts of the FOGI rates
SEED = 3122
np.random.seed(SEED)

max_stochastic_error_rate = 0.01
hamiltonian_error_var = 0.05
ar = mdl_datagen.fogi_errorgen_components_array(include_fogv=False, normalized_elem_gens=True)


# add hamiltonian noise
ar[0:3] = np.random.normal(0, hamiltonian_error_var, 3)
ar[9] = np.random.normal(0, hamiltonian_error_var)
ar[6] = np.random.normal(0, hamiltonian_error_var)
mdl_datagen.set_fogi_errorgen_components_array(ar, include_fogv=False, normalized_elem_gens=True)
print('hamiltonian-only MSE with target', mserror(mdl_datagen, target_model))
print('hamiltonian-only agsi with target', avg_gs_infidelity(mdl_datagen, target_model))

# add stochastic noise
ar[3:6] = max_stochastic_error_rate*np.random.rand(3)
ar[7:9] = max_stochastic_error_rate*np.random.rand(2)
ar[10:12] = max_stochastic_error_rate*np.random.rand(2)
mdl_datagen.set_fogi_errorgen_components_array(ar, include_fogv=False, normalized_elem_gens=True)

print('MSE with target', mserror(mdl_datagen, target_model))
print('agi with target', avg_gs_infidelity(mdl_datagen, target_model))

hamiltonian-only MSE with target 0.013944923672360084
hamiltonian-only agsi with target 0.0027790049441692053
MSE with target 0.014171703875716937
agi with target 0.00932546039819362


In [220]:
mfs = MatrixForwardSimulator(target_model)

In [221]:
print('model is cptp', model_is_cptp(mdl_datagen))

model is cptp True


In [222]:
germs = std.germs()
mfids = std.meas_fiducials()
pfids = std.prep_fiducials()

In [223]:
class ExtendedKalmanFilter():
    """
    An extended Kalman filter for gate-set tomography
    
    --- Parameters ---
    model: an underlying pygsti model
    num_params: number of parameters in the pygsti model
    P: current covariance matrix
    """
    def __init__(self, model, P0):
        self.model = model.copy()
        self.P = P0
        self.param_history = [self.model.to_vector()]
        self.covar_history = [self.P]
        
    def update(self, circ, count_vec, clip_range=[-1,1], Q=None, R_additional=None, max_itr=1, itr_eps=1e-4):
        """
        Makes an exact update to the model
        where the jacobian is calculated as the current estimate
        
        --- Arguments ---
        circ: pygsti circuit used in the update
        count_vec: vector of observed counts
        clip_range: reasonable clipping range for the parameter update
        Q: state-space covariance 
        R_additional: additional measurement covariance
        max_itr: maximum number of iterations to the update
        itr_eps: epsilon for minimum difference to end iterated updates
        
        --- Returns --- 
        innovation: the prior innovation
        kgain: the Kalman gain
        """
        prior_covar = self.P
        prior_state = self.model.to_vector()
        hilbert_dims = 2**(circ.width)
        
        # find the predicted frequency for the circuit outcome under the model
        probs = self.model.probabilities(circ)
        p_model = vector_from_outcomes(probs, hilbert_dims)

        # calculate the observed frequency
        total_counts = sum(count_vec)
        observation = count_vec/total_counts

        # calculate jacobian
        jacob = matrix_from_jacob(self.model.sim.dprobs(circ), 2**circ.width)

        # calculate the covaraiance of the observation
        mean_frequency = ( count_vec+np.ones(len(count_vec)) )/( sum(count_vec)+len(count_vec) )
        R = (1/(sum(count_vec)+len(count_vec)+1))*categorical_covar(mean_frequency)

        # add any additional noise
        if R_additional is not None:
            R += R_additional
        if Q is None: 
            Q = 0*np.eye(self.model.num_params)

        # Kalman gain
        P = prior_covar + Q
        kgain = P@jacob.T@np.linalg.pinv(jacob@P@jacob.T + R, 1e-15)

        # Kalman update
        innovation = observation - p_model
        post_state = prior_state + kgain@innovation
        post_state = np.clip(post_state, clip_range[0], clip_range[1])
            

        
        # update class parameters
        self.P = (np.eye(self.model.num_params) - kgain@jacob)@P
        self.model.from_vector(post_state)
        
        self.param_history.append(post_state)
        self.covar_history.append(self.P)
        
        return innovation, kgain 
    
    

In [224]:
germs

[Circuit([]@(0)),
 Circuit(Gxpi2:0@(0)),
 Circuit(Gypi2:0@(0)),
 Circuit(Gxpi2:0Gypi2:0@(0)),
 Circuit(Gxpi2:0Gxpi2:0Gypi2:0@(0))]

In [225]:
germs[3]

Circuit(Gxpi2:0Gypi2:0@(0))

In [226]:
target_model.num_params

18

In [209]:
for g in germs:
    dgerm = mfs.dproduct(g)
    print(dgerm.shape)
    gmat = dgerm.reshape(dgerm.shape[0], -1)
    dpd = np.conj(gmat.T)@P@gmat
    print(np.trace(np.linalg.pinv(dpd)))

(18, 4, 4)
1190.7532199931695


ValueError: operands could not be broadcast together with shapes (16,0) (16,6) (16,0) 

1190.7532199931695

In [111]:
dgerm.T.shape

(4, 4, 18)

In [93]:
mse = mserror(target_model, mdl_datagen)
initial_covar = (mse/target_model.num_params)*np.eye(target_model.num_params)

In [94]:
ekf = ExtendedKalmanFilter(target_model, initial_covar)

In [95]:
P = ekf.P

In [102]:
np.trace(P)

0.014171703875716936

In [103]:
np.linalg.det(P)

1.3511144839590893e-56

In [126]:
P.shape

(18, 18)

In [127]:
dgerm.shape

(18, 4, 4)

In [112]:
for g in germs:
    dgerm = mfs.dproduct(g)
    np.tensordot(dgerm.T, P)

ValueError: shape-mismatch for sum